In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sumit\\OneDrive\\Desktop\\Chest Cancer Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sumit\\OneDrive\\Desktop\\Chest Cancer Classification'

In [5]:
import tensorflow as tf

In [6]:
import dagshub
dagshub.init(repo_owner='sumitprasadd.wb24', repo_name='Chest-Cancer-Classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as sumitprasadd.wb24

Initialized MLflow to track repo "sumitprasadd.wb24/Chest-Cancer-Classification"

Repository sumitprasadd.wb24/Chest-Cancer-Classification initialized!

🏃 View run delicate-pig-23 at: https://dagshub.com/sumitprasadd.wb24/Chest-Cancer-Classification.mlflow/#/experiments/0/runs/2c0feb1d52e24fb39c2ebd49a8f40ddd
🧪 View experiment at: https://dagshub.com/sumitprasadd.wb24/Chest-Cancer-Classification.mlflow/#/experiments/0


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from src.ChestCancerClassification.constants import *
from src.ChestCancerClassification.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/data",
            mlflow_uri="https://dagshub.com/sumitprasadd.wb24/Chest-Cancer-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 93 images belonging to 2 classes.
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.9785 - loss: 0.0987


2026/02/13 16:14:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/13 16:14:05 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'VGG16Model'.
2026/02/13 16:15:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run fun-kit-101 at: https://dagshub.com/sumitprasadd.wb24/Chest-Cancer-Classification.mlflow/#/experiments/0/runs/2bd118468cc348408c20b1bfdea7102b
🧪 View experiment at: https://dagshub.com/sumitprasadd.wb24/Chest-Cancer-Classification.mlflow/#/experiments/0
